In [72]:
import numpy as np
from scipy.optimize import fsolve
from scipy.special import lambertw

ID_hxn_connect = .824 #in
g_imp = 32.2
g_met = 9.8

Thotin = 40 #C
Tcoldin = 25 #C
Vhotin = 25 #lpm
Vhotin_met = 0.0004166667 #m^3/s
rho_water_m = 992 #kg/m^3
MhotinSI = Vhotin_met*rho_water_m #kg/s
cp_water_m = 4.18 #kj/kg/C
heat_req = 11 #kw
deltaThot = heat_req/(MhotinSI*cp_water_m) #C
Thotout = Thotin -  deltaThot #C
epsilon = .575 #roughness!

#rho_water = 62.4
mu_water = .000653 #pa*s
nu_water = .000000658

def reynolds_f(density,D,vel,mu): #ASSUMPTION: D in m and vel in m/s
    return density*D*vel/mu

def Tlogmean(hotin,coldin,hotout,coldout):
    returnable = ((hotin-coldout)-(hotout-coldin))/np.log((hotin-coldout)/(hotout-coldin))
    #print("Tlogmean:",returnable)
    return returnable

def colebrook(velocity,diameter): #assume in SI?
    radius = diameter/4
    b = epsilon/14.8/radius
    re = rho_water_m*velocity*diameter/mu_water
    a = 2.51/re
    f = (2*np.real(lambertw(np.log(10)/2/a*10**(b/2/a)))/ln(10)-b/a)**-2
    return f

class heat_ex:
    def __init__(self,SA,PL,k24,k31,nop):
        self.SA = SA
        self.PL = PL
        self.k24 = k24
        self.k31 = k31
        self.nop = nop
        self.pipe_D = ID_hxn_connect*.0254 #m
        self.pipe_A = self.pipe_D**2/4*np.pi #m^2
    def set_flow(self,Vdot): #ASSUMPTION: Vdot is in lpm
        #print("Vdot",Vdot)
        #print("Vdot:",Vdot)
        self.Vdot = Vdot*0.0000166667 #m^3/s
        self.massdot = self.Vdot*rho_water_m #kg/s
        #print(self.pipe_A,self.Vdot)
        self.velocity = self.Vdot/self.pipe_A #m/s
    def presdrop_3_1(self):
        self.presdrop = self.k31*self.velocity**2/(2*g_met) #meters
    def calc_re(self):
        cur_vel = min(Vhotin_met/self.pipe_A,self.velocity)
        self.re = reynolds_f(rho_water_m,self.pipe_D,cur_vel,mu_water) #This is the reynolds for U calculations only!!!
        #print("Re",self.re)
        #AKA, this will be the reynolds for the lowest flowrate stream
    def calc_U(self):
        self.U = self.re/self.nop+1800 #W/m^2/K
        #print("U", self.U)
        #print("re",self.re)
        #print("U",self.U)
    def calc_Tcoldout(self):
        deltaTcold = heat_req/(self.massdot*cp_water_m) #C
        self.Tcoldout = Tcoldin+deltaTcold #C
        #print("mass/delta/cold",self.massdot,deltaTcold,self.Tcoldout)
    def Qdif(self):
        self.Q = -self.U*self.SA*Tlogmean(Thotin,Tcoldin,Thotout,self.Tcoldout)
        #print("Q",self.Q)
        self.dif = (self.Q+heat_req*1000)
        #print(self.Q,heat_req*1000)
    def calcs(self):
        self.calc_re()
        self.calc_U()
        self.calc_Tcoldout()
        self.Qdif()
    
        
hxn1210 = heat_ex(.12,7.5,209,102,10)
hxn1240 = heat_ex(.48,7.5,9.65,8.93,40)
hxn2320 = heat_ex(.46,12,42.8,36.4,20)
hxn2340 = heat_ex(.92,12,12.5,13.1,40)
def fsolve_hxn(flowrate,hxn):
    hxn = hxn[0]
    hxn.set_flow(flowrate)
    hxn.presdrop_3_1()
    hxn.calcs()
    returnable = hxn.dif
    return returnable

#hxn1210.set_flow(20)
#hxn1210.calcs()
#print(hxn1210.Tcoldout)
#hxn1210.Qdif()
optimalV1210 = fsolve(fsolve_hxn,15,args=[hxn1210])
optimalV1240 = fsolve(fsolve_hxn,15,args=[hxn1240])
optimalV2320 = fsolve(fsolve_hxn,15,args=[hxn2320])
optimalV2340 = fsolve(fsolve_hxn,15,args=[hxn2340])
#print(optimalV1210)
#print(optimalV1240)
#print(optimalV2320)
#print(optimalV2340)
#print(hxn1210.Tcoldout)

#print(hxn1210.presdrop)
print(optimalV1240)
print(hxn1240.presdrop)
print(optimalV2320)
print(hxn2320.presdrop)
print(optimalV2340)
print(hxn2320.presdrop)
someth = lambertw(optimalV1240[0])
print(np.real(someth))
#print(hxn2340.Tcoldout)

[23.66486328]
[0.59879635]
[18.40039035]
[1.47562208]
[13.06246778]
[1.47562208]
2.321694205782145
